In [ ]:
import pandas as pd
import numpy as np
import re

# Advanced REGEX

Como vimos na aula de `strings` muitas vezes a forma mais simples de manipularmos um `string` é utilizando padrões REGEX.

A utilização de REGEX é calcada na construção de **padrões**: ao invés de definir buscas simples (como quando procuramos uma sequência específica de caractéres), o REGEX nos permite construir padrões flexíveis, capaz de *encontrar*  sub-strings distintos. Já vimos como construir alguns padrões simples: hoje faremos uma revisão rápida dos conceitos básicos de REGEX e introduziremos novos conceitos, como grupos e os qualificadores para fim e começo.

## Revisão de Expressões Regulares

https://regexr.com/

O aspecto fundamental para a utilização de REGEX é a construção do padrão de busca. Vamos revisar alguns 

In [ ]:
texto = '''
        Quando certa manhã Gregor Samsa acordou de sonhos intranquilos,
        encontrou-se em sua cama metamorfoseado num inseto monstruoso.
        '''

### Padrões e Conjuntos

A forma mais simples de utilizarmos um padrão é através de uma busca por *sub-string*. Padrões que não contém conjuntos ou carácteres especiais são chamados de **padrões literais**.

#### Busca Literal

In [ ]:
print(re.findall('Samsa', texto))

In [ ]:
print(re.findall('so', texto))

In [ ]:
print(re.findall('SO', texto))

In [ ]:
print(re.findall('samsa', texto))

In [ ]:
print(re.findall('metamorfose', texto))

#### Conjuntos

Podemos utilizar conjuntos para expandir nossa capacidade de busca. Vamos começar construindo um conjunto através do operador OR (`|`).

##### Exemplo 1: `m|e|t|a|m|o|r|f|o|s|e`

In [ ]:
print(re.findall('m|e|t|a|m|o|r|f|o|s|e', texto))

In [ ]:
print(re.findall('[metamorfose]', texto))

In [ ]:
re.findall('[metamorfose]', texto) == re.findall('m|e|t|a|m|o|r|f|o|s|e', texto)

##### Exemplo 2: Cidade de São Paulo

In [ ]:
text = 'São Paulo Sao Paulo Sáo Paulo Sun Paulo seu paulo san paolo sao paulo são paolo sAo Paolo sao_paulo'

pattern = r'[Ss][ãaáàâAÃÁÀâeu][oun][ _][Pp]a[uo]lo'
re.findall(pattern, text)

In [ ]:
re.sub(pattern, 'São Paulo', text)

In [ ]:
nomes_sp = ['São Paulo', 'Sao Paulo', 'Sáo Paulo', 
            'Sun Paulo', 'seu paulo', 'san paolo', 
            'sao paulo', 'são paolo', 'sAo Paolo', 
            'sao_paulo', 'Rio de Janeiro']
print(re.sub(pattern, 'São Paulo', nomes_sp[0]))

In [ ]:
print(re.sub(pattern, 'São Paulo', nomes_sp[1]))

In [ ]:
print(re.sub(pattern, 'São Paulo', nomes_sp))

In [ ]:
nome_sp_limpo = [re.sub(pattern, 'São Paulo', nome) for nome in nomes_sp]
print(nome_sp_limpo)

In [ ]:
tb_nomes = pd.DataFrame({'nome' : nomes_sp})
tb_nomes.head()

In [ ]:
tb_nomes['nome'].map(lambda x: re.sub(pattern, 'São Paulo', x))

Qualquer coisa entre `[]`, em padrão REGEX, é um conjunto! Pensando nos strings de forma posicional, cada conjunto ocupa **apenas uma posição do string**! Por exemplo, o padrão `r'[Ss][AaÃãÂâÁáÀà]` tem comprimento dois: procurando `[Ss]` na primeira posição e `[AaÃãÂâÁáÀà]` na segunda!

##### Atalhos para Conjunto

Podemos utilizar a notação `r'[A-D]'` para construir padrões contendo todos os carácteres entre duas letras.

In [ ]:
lista_tarefas = '''
    A) cortar grama
    B) arrumar porta
    C) instalar calha
    D) ligar para Pedro as 9
    '''

In [ ]:
re.findall(r'A|B|C|D', lista_tarefas)

In [ ]:
re.findall(r'[ABCD]', lista_tarefas)

In [ ]:
re.findall(r'[A-D]', lista_tarefas)

In [ ]:
re.findall(r'[A-Z]', lista_tarefas)

In [ ]:
lista_tarefas = '''
    1) cortar grama
    2) arrumar porta
        2a trocar fechadura
    3) instalar calha
    4) ligar para Pedro as 9 9983
    '''
re.findall('1|2|3|4', lista_tarefas)

In [ ]:
re.findall(r'[1234]', lista_tarefas)

In [ ]:
re.findall(r'[1-4]', lista_tarefas)

In [ ]:
re.findall(r'[0-9]', lista_tarefas)

In [ ]:
re.findall(r'[0-9A-Z]', lista_tarefas)

In [ ]:
re.findall(r'[0-9][a-z]', lista_tarefas)

Os atalhos de conjunto mais úteis são:

* [a-z]: Qualquer letra minúscula;
* [A-Z]: Qualquer letra maiúscula;
* [0-9]: Qualquer digito.

#### Classes de Carácteres

As classes de carácteres são *atalhos* para conjuntos comuns:

* `\d`: carácteres numéricos;
* `\w`: carácteres alfa-numéricos;
* `\s`: espaços;
* `\D`: carácteres não-numéricos.

In [ ]:
text = 'aoijo (  $ p io x -o = 3232 13 ™¡¡™£¡Ωå 3.1 áéóãà'
pattern = r'\d'
print(re.findall(pattern, text))

In [ ]:
text = 'aoijo (  $ p io x -o = 3232 13 ™¡¡™£¡Ωå 3.1 áéóãà'
pattern = r'[\D]'
print(re.findall(pattern, text))

## Quantificadores 

Assim como os conjuntos tornam os caracteres de uma posição flexíveis (`r'[Aa]'` encontra tanto `A` quanto `a` na primeira posição), os quantificadores tornam o número de posições que um conjunto (ou caractere) ocupam.

* *: Encontra o caractere (ou conjunto) aterior 0 ou mais vezes consecutivas;
* +: Encontra o caractere (ou conjunto) anterior 1 ou mais vezes consecutivas;
* ?: Encontra o caractere (ou conjunto) anterior 0 ou 1 vez.

Por exemplo, o padrão `r'a+'` encontra `'a'`, `'aa'`, `'aaa'`, etc...

In [ ]:
text = 'a aa aaa aaaa aaaaa aaaaaa'
pattern = r'a+'
print(re.findall(pattern, text))

Já o padrão `r'a*'` encontra `''`, `'a'`, `'aaa'`, etc...

In [ ]:
text = 'a aa aaa aaaa aaaaa aaaaaa'
pattern = r'a*'
print(re.findall(pattern, text))

Por fim, o padrão `r'ab?a*'` encontra todos os *sub-strings* que começam com `a`, possivelmente são seguidos de 1 `b` e podem ter múltiplos `a`s no final:

1. `a`;
1. `ab`;
1. `aba`;
1. `aa`;
1. `abaa`;
1. `...`

In [ ]:
text = 'a aba baaa aaaba aaaaa aaaaaa'
pattern = r'ab?a*'
print(re.findall(pattern, text))

Porque o padrão acima separou `aaaba` em `aaa` e `a`?

### Exemplo - Encontrando números

In [ ]:
text = '3.1 3,3 45,3 1000 0'
pattern = r''
print(re.findall(pattern, text))

## Quantificadores Especiais
O quantificador `{n}` funciona como um `+` controlado: podemos especificar quantas vezes queremos encontrar o caractere (ou conjunto) precendente:

* {n} : Exatamente n-vezes;
* {n,} : Pelo menos n-vezes;
* {n,m} : Entre n e m vezes;

In [ ]:
text = 'a aba baaa aaaba aaaaa aaaaaa'
pattern = r'a{1}'
print(re.findall(pattern, text))

In [ ]:
text = 'a aba baaa aaaba aaaaa aaaaaa'
pattern = r'a{2}'
print(re.findall(pattern, text))

In [ ]:
text = 'a aba baaa aaaba aaaaa aaaaaa'
pattern = r'a{4}'
print(re.findall(pattern, text))

In [ ]:
text = 'a aba baaa aaaba aaaaa aaaaaa'
pattern = r'a{2,}'
print(re.findall(pattern, text))

In [ ]:
text = 'a aba baaa aaaba aaaaa aaaaaa'
pattern = r'a{2,3}'
print(re.findall(pattern, text))

#### Exemplo - Encontrando CPFs


Vamos construir um padrão para encontrar número com formato de CPF em um string.

In [ ]:
text = '339.211.273-23 33921127323 339.211.27323 119730 R$13542 43.544.23023'
pattern = '[0-9]{1,3}\.?[0-9]{3}\.?[0-9]{3}-?[0-9]{1,2}'
print(re.findall(pattern, text))

## Meta-caracteres

Meta-caracteres são caracteres *especiais*: o REGEX não os interpreta de forma literal. Se quisermos utilizar um meta-caractere literalmente (como o `.` no exemplo do CPF) devemos escapá-lo com `\`.

* `.` : Qualquer caractere exceto newline (`\n`);
* `[^]` : **Dentro de um conjunto** representa a negação (inverte o conjunto);
* `^`: **Fora de um conjunto** representa o começo da linha;
* `$` : Fim da linha;
* `|` : Operador OU;

### Limpando newlines com `.`

O meta-caractere `.` pode ser utilizado para limparmos os `\n` de um string:

In [ ]:
text = '''My boss asked me to turn in my TPS reports. 
I told him they were done, but they are not.'''
pattern = r'.'
print(re.findall(pattern, text))

In [ ]:
print(text)

In [ ]:
print(''.join(re.findall(pattern, text)))

### Negando Conjuntos

Dentro de um conjunto, o caractere `[^]` representa a negação do conjunto (encontramos tudo **QUE NÃO ESTÁ NO CONJUNTO**). Muitas vezes é mais fácil especificar **O QUE NÃO QUEREMOS** do que o que queremos!

In [ ]:
text = """My boss asked me to turn in my TPS reports. 
I told him they were done, but they are not."""
pattern = r'[^a-mA-M]'
print(re.findall(pattern, text))

In [ ]:
print(''.join(re.findall(pattern, text)))

### Encontrando padrões no começo ou fim do string

Os caracteres `^` e `$` nos permitem encontrar o começo ou fim, respectivamente, de um string.

In [ ]:
text = '''My boss asked me to turn in my TPS reports.
The boss told him they were done, but they are not.'''

In [ ]:
pattern = r'^My boss'
re.findall(pattern, text)

In [ ]:
pattern = r'^The boss'
re.findall(pattern, text)

O padrão do REGEX no Python é considerar o começo e fim do **string** como um todo. Podemos alterar esse padrão para que eles encontrem o começo e fim de cada nova linha (criada com `\n`).

In [ ]:
pattern = r'^The boss'
re.findall(pattern, text, re.MULTILINE)

In [ ]:
pattern = r'are not.$'
re.findall(pattern, text)

In [ ]:
pattern = 'reports.$'
re.findall(pattern, text, re.MULTILINE)

### Aplicando `$` com `*`

In [ ]:
text = '''My boss asked me to turn. in my TPS reports.
My boss told him they were done, but they are not.'''

In [ ]:
re.findall(r'are not.$', text)

In [ ]:
re.findall(r'.are not.$', text)

In [ ]:
re.findall(r'.*are not.$', text)

In [ ]:
re.findall(r'.*\.$', text, re.MULTILINE)

## Ganância (Greediness)
https://docs.python.org/3/howto/regex.html#greedy-versus-non-greedy

In [ ]:
text = 'You are yelling! So I will yell too! Let me yell!'

In [ ]:
pattern = r'.*!'
print(re.findall(pattern, text))

When repeating a regular expression, as in a*, **the resulting action is to consume as much of the pattern as possible.**

In [ ]:
pattern = r'[ a-zA-Z]*!'
print(re.findall(pattern, text))

## Utilizando `grupos`

Até agora, utilizamos padrões para extrair substrings completos. Muitas vezes, no entanto, queremos utilizar um REGEX para extrair múltiplas informações de um mesmo string a partir de uma estrutura determinada. Para isto, usaremos grupos!

https://docs.python.org/3/howto/regex.html#grouping

In [ ]:
text = '''
From: author@example.com
User-Agent: Thunderbird 1.5.0.9 (X11/20061227)
MIME-Version: 1.0
To: editor@example.com
'''

In [ ]:
pattern = r'(.*):(.*)'
re.findall(pattern, text)

O resultado de um REGEX feito a partir de um padrão com grupos é uma lista de tuplas. Cada elemento da lista corresponde à um match do padrão completo e cada elemento da tupla corresponde à um grupo do padrão (mesmo que este grupo esteja vazio):

In [ ]:
text = '''
From:
User-Agent: Thunderbird 1.5.0.9 (X11/20061227)
MIME-Version: 1.0
To: editor@example.com
'''
pattern = r'(.*):(.*)'
re.findall(pattern, text)

### Exemplo - Separação de Códgio Internacional, DDDs e Telefones

Uma tarefa comum que encontramos no tratamento de `strings` é a separação de um string semi-estruturado (por exemplo, um telefone) em seus componentes.

In [ ]:
lista_telefone = ['+55(19)35613675', '+55(11)29934999', '+1(678)818977222', '+1(544)932226172']

In [ ]:
pattern = r'\+([0-9]*)\(([0-9]*)\)([0-9]*)'
[re.findall(pattern, telefone) for telefone in lista_telefone]